In [ ]:
from datasets import load_dataset
NUM_LABELS = 20
# print(len(combined_dataset))

id2label = {
    0: "alt.atheism",
    1: "comp.graphics",
    2: "comp.os.ms-windows.misc",
    3: "comp.sys.ibm.pc.hardware",
    4: "comp.sys.mac.hardware",
    5: "comp.windows.x",
    6: "misc.forsale",
    7: "rec.autos",
    8: "rec.motorcycles",
    9: "rec.sport.baseball",
    10: "rec.sport.hockey",
    11: "sci.crypt",
    12: "sci.electronics",
    13: "sci.med",
    14: "sci.space",
    15: "soc.religion.christian",
    16: "talk.politics.guns",
    17: "talk.politics.mideast",
    18: "talk.politics.misc",
    19: "talk.religion.misc"
}

label2id = {
    "alt.atheism": 0,
    "comp.graphics": 1,
    "comp.os.ms-windows.misc": 2,
    "comp.sys.ibm.pc.hardware": 3,
    "comp.sys.mac.hardware": 4,
    "comp.windows.x": 5,
    "misc.forsale": 6,
    "rec.autos": 7,
    "rec.motorcycles": 8,
    "rec.sport.baseball": 9,
    "rec.sport.hockey": 10,
    "sci.crypt": 11,
    "sci.electronics": 12,
    "sci.med": 13,
    "sci.space": 14,
    "soc.religion.christian": 15,
    "talk.politics.guns": 16,
    "talk.politics.mideast": 17,
    "talk.politics.misc": 18,
    "talk.religion.misc": 19
}
dataset = load_dataset("json", data_files="train_dataset_20_newsgroups_noisy.json", split="train")
train_texts=[]
train_labels = []
for stuff in dataset:
    train_texts.append(stuff["messages"][1]["content"])
    train_labels.append(id2label[int(stuff["messages"][2]["content"])])

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset("json", data_files="test_dataset_20_newsgroups.json", split="train")
test_texts=[]
test_labels = []
for stuff in dataset:
    test_texts.append(stuff["messages"][1]["content"])
    test_labels.append(id2label[int(stuff["messages"][2]["content"])])

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download("wordnet")

# Step - a : Remove blank rows if any.
train_texts = [text for text in train_texts if text.strip()]

# Step - b : Change all the text to lower case.
train_texts = [text.lower() for text in train_texts]

# Step - c : Tokenization
train_texts = [[word for word in word_tokenize(text)] for text in train_texts]

# Step - d : Remove Stop words, Non-Numeric and perform Word Stemming/Lemmatization
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

train_texts_final = []
for entry in train_texts:
    final_words = []
    word_lemmatizer = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_final = word_lemmatizer.lemmatize(word, tag_map[tag[0]])
            final_words.append(word_final)
    train_texts_final.append(final_words)

[nltk_data] Downloading package punkt to /home/trainee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/trainee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/trainee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/trainee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download("wordnet")

# Step - a : Remove blank rows if any.
test_texts = [text for text in test_texts if text.strip()]

# Step - b : Change all the text to lower case.
test_texts = [text.lower() for text in test_texts]

# Step - c : Tokenization
test_texts = [[word for word in word_tokenize(text)] for text in test_texts]

# Step - d : Remove Stop words, Non-Numeric and perform Word Stemming/Lemmatization
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

test_texts_final = []
for entry in test_texts:
    final_words = []
    word_lemmatizer = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_final = word_lemmatizer.lemmatize(word, tag_map[tag[0]])
            final_words.append(word_final)
    test_texts_final.append(final_words)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(train_labels)
Test_Y = Encoder.fit_transform(test_labels)

In [ ]:
train_texts_final

In [ ]:

Tfidf_vect = TfidfVectorizer(max_features=5000)

# Flatten the train_texts_final and test_texts_final lists
train_texts_flat = [' '.join(text) for text in train_texts_final]
test_texts_flat = [' '.join(text) for text in test_texts_final]

Tfidf_vect.fit(train_texts_flat)
Train_X_Tfidf = Tfidf_vect.transform(train_texts_flat)
Test_X_Tfidf = Tfidf_vect.transform(test_texts_flat)

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score (Noisy)-> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score (Clean) ->  88.58038513210927


In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score (Noisy)-> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score (Clean) ->  85.7590685176892


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'C': [1, 10],
          'gamma': [0.001, 0.01, 1]}
model = svm.SVC()
grid = GridSearchCV(estimator=model, param_grid=parameters)
grid.fit(Train_X_Tfidf, Train_Y)
predictions_SVM_grid =grid.predict(Test_X_Tfidf)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_)

print("--------------------------")
print(f"SVM Grid Acc ->{accuracy_score(predictions_SVM_grid,Test_Y)*100}")